# Wykład 10: Uczenie sieci neuronowych z Keras'em

### Cel: 
- Nauka tworzenia i uczenia sieci neuronowych z `tensorflow` i Keras'em

### Zbiór danych:
- Do testów wykorzystamy zbiór ręcznie pisanych cyfr
- http://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_digits.html#sklearn.datasets.load_digits

In [ ]:
%matplotlib inline 
import matplotlib.pyplot as plt
import numpy as np
from sklearn.datasets import load_digits

digits = load_digits()

In [ ]:
i = 42
plt.imshow(digits.images[i], interpolation='nearest')
plt.title("label: %d" % digits.target[i]);

## Podział danych na treningowe i testowe

Tak jak w poprzednich zadaniach z regresji zrobimy podział na dane treningowe wykorzystywane do nauki sieci
oraz testowe i na tak przygotowanych danych będziemy sprawdzać nasz model.

In [ ]:
from sklearn.model_selection import train_test_split


data = np.asarray(digits.data, dtype='float32')
target = np.asarray(digits.target, dtype='int32')

X_train, X_test, y_train, y_test = train_test_split(
    data, target, test_size=0.20, random_state=17)

In [ ]:
i = 142
plt.imshow(X_train[i].reshape(8, 8), interpolation='nearest')

## Wstępne przygotowanie danych wejściowych (preprocessing)

Chcemy aby dane wejściowe (nasze cyfry) miały w przybliżeniu takie same parametry i znormalizowane kolory.

In [ ]:
from sklearn import preprocessing

# wykorzystamy funkcje StandardScaler
help(preprocessing.StandardScaler)
# lub
preprocessing.StandardScaler?

In [ ]:
scaler = preprocessing.StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# print(scaler.mean_)
# print(scaler.scale_)

Zobaczmy jak teraz wyglądają nasze dane

In [ ]:
i = 142
plt.imshow(X_train[i].reshape(8, 8), interpolation='nearest')

Obiekt `scaler` jest operacją odwracalną, która umożliwia nam odtworzenie oryginalnego obrazu

In [ ]:
plt.imshow(scaler.inverse_transform(X_train[i]).reshape(8, 8), interpolation='nearest')

In [ ]:
print(X_train.shape, y_train.shape)

In [ ]:
print(X_test.shape, y_test.shape)

## Wstępne przygotowanie danych docelowych (naszych etykiet)

Aby odpowiednio nauczyć naszą sieć neuronową musimy przetworzyć etykiety do odpowiedniego formatu.
Zobaczmy najpierw jak wygląda nasz wektor `y_train`. Jak widać są to po prostu liczby całkowite.

In [ ]:
y_train[:42]

Konwersja danych do formatu wykorzystywanego przez naszą sieć neuronową. Skorzystamy z funkcji
Keras'a `to_categorial`. Na przykład cyfrę `9` zamieniamy na wektor `[0., 0., 0., 0., 0., 0., 0., 0., 0., 1.]`. Tak będziemy kategoryzować nasze ręcznie pisane cyfry, czyli warstwa wyjściowa będzie miała 10 neuronów (jeden dla każdej cyfry).

In [ ]:
from tensorflow.keras.utils import to_categorical

Y_train = to_categorical(y_train)
Y_train[:7]

## Zaprogramujemy teraz nasza sieć neuronową z wykorzystaniem Keras'a

Naszym celem jest:

- Budowanie i trening sieci `feedforward` z `Kerasem`
    - https://www.tensorflow.org/guide/keras/overview
- Eksperymentowanie z różnymi algorytmami uczenia (optimizers), funkcjami aktywacji, wielkościami warstw i inicjalizacjami wag

### Model sieci neuronowej z wykorzystaniem Keras'a

Wykorzystamy wysoko-poziomowe API Kerasa

- na początku definiujemy nasz model jako kolejne warstwy sieci (o odpowiednich wymiarach)
- wybieramy odpowiednią funkcję kosztu (loss) i metodę uczenia (optimizer) SGD (stochastic gradient descent)
- następnie przekazujemy naszemu modelowi dane trenujące i ustalamy liczbę kroków uczenia (epoch)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras import optimizers

input_dim = X_train.shape[1]  # warstwa wejściowa
hidden_dim = 100              # warstwa ukryta
output_dim = 10               # warstwa wyjściowa - 10 neuronów (każda cyfra)

model = Sequential()
model.add(Dense(hidden_dim, input_dim=input_dim, activation="tanh"))
# przy klasyfikacji bardzo często w warstwie wyjściowej wybieramy funkcje aktywacji 'softmax'
model.add(Dense(output_dim, activation="softmax"))

model.compile(optimizer=optimizers.SGD(lr=0.1),
              loss='categorical_crossentropy', metrics=['accuracy'])

history = model.fit(X_train, Y_train, validation_split=0.2, epochs=15, batch_size=32)

### Wizualizacja procesu uczenia (zbieżność)

In [ ]:
history.history

In [ ]:
history.epoch

Wykorzystajmy `pandas` do łatwiejszego przetwarzania danych

In [ ]:
import pandas as pd

history_df = pd.DataFrame(history.history)
history_df["epoch"] = history.epoch
history_df

In [ ]:
fig, (ax0, ax1) = plt.subplots(nrows=2, sharex=True, figsize=(12, 6))
history_df.plot(x="epoch", y=["loss", "val_loss"], ax=ax0)
history_df.plot(x="epoch", y=["accuracy", "val_accuracy"], ax=ax1);

### Monitorowanie procesu uczenia z wykorzystaniem `Tensorboard`

In [ ]:
%load_ext tensorboard

In [ ]:
!rm -rf tensorboard_logs

In [ ]:
import datetime
from tensorflow.keras.callbacks import TensorBoard

model = Sequential()
model.add(Dense(hidden_dim, input_dim=input_dim, activation="tanh"))
model.add(Dense(output_dim, activation="softmax"))

model.compile(optimizer=optimizers.SGD(lr=0.1),
              loss='categorical_crossentropy', metrics=['accuracy'])

timestamp =  datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
log_dir = "tensorboard_logs/" + timestamp
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

model.fit(x=X_train, y=Y_train, validation_split=0.2, epochs=15,
          callbacks=[tensorboard_callback]);

In [ ]:
%tensorboard --logdir tensorboard_logs

### Przetestujemy różne algorytmy uczenia (optimizers)

- zmniejszymy współczynnik uczenia (learning rate) o 10 lub 100. Co można zaobserwować?

- zwiększymy współczynnik uczenia (learning rate) i zobaczymy jak uczenie nie jest już zbieżne.

- Skonfigurujemy SGD tak aby wykorzystać moment Nesterow o wartości np. 0.9
  
**Dokumentacja**: 

Keras API: https://www.tensorflow.org/api_docs/python/tf/keras

Można też skorzystać z dokumentacji z poziomu notatnika

```python
optimizers.SGD?
```

Przypominam, że mamy też podpowiedzi przez "shift-tab" np.

```python
optimizers.SGD(<shift-tab>
```

In [ ]:
optimizers.SGD?

## Analiza

- zauważmy, że ustawiając współczynnik uczenia na małą wartość (np. lr=0.001) powoduje bardzo wolny proces uczenia, w przypadku naszych danych nie mamy zbieżności po 15 epokach.

- wykorzystując moment można "złagodzić" mały współczynnik uczenia (przynajmniej trochę ...)

- natomiast ustawiając współczynnik uczenia na dużą wartość (np. lr=10) powoduje losowe krążenie w obszarze dobrego minimum i uniemożliwia osiągnięcie funkcji celu (loss) nawet po 30 epokach.

In [ ]:
model = Sequential()
model.add(Dense(hidden_dim, input_dim=input_dim,
                activation="tanh"))
model.add(Dense(output_dim, activation="softmax"))
model.add(Activation("softmax"))

optimizer = optimizers.SGD(lr=0.1, momentum=0.9, nesterov=True)
model.compile(optimizer=optimizer, loss='categorical_crossentropy',
              metrics=['accuracy'])
history = model.fit(X_train, Y_train, validation_split=0.2,
                    epochs=15, batch_size=32)

fig, (ax0, ax1) = plt.subplots(nrows=2, sharex=True, figsize=(12, 6))
history_df = pd.DataFrame(history.history)
history_df["epoch"] = history.epoch
history_df.plot(x="epoch", y=["loss", "val_loss"], ax=ax0)
history_df.plot(x="epoch", y=["accuracy", "val_accuracy"], ax=ax1);



# Inne metody uczenia

- zastąpimy SGD przez algorytm uczenia Adam i uruchomimy z domyślnymi parametrami. Pamiętaj, że mamy też
    `tab-complete` dokładnie `optimizers.<TAB>`
    
- dodamy jeszcze jedną warstwę i wykorzystamy ReLU dla każdej warstwy ukrytej. Czy dalej możemy wykorzystać model Adam z domyślnymi parametrami uczenia?

In [ ]:
model = Sequential()
model.add(Dense(hidden_dim, input_dim=input_dim,
                activation="relu"))
model.add(Dense(hidden_dim, activation="relu"))
model.add(Dense(output_dim, activation="softmax"))

optimizer = optimizers.Adam(lr=0.001)
model.compile(optimizer=optimizer, loss='categorical_crossentropy',
              metrics=['accuracy'])

history = model.fit(X_train, Y_train, validation_split=0.2,
                    epochs=15, batch_size=32)
fig, (ax0, ax1) = plt.subplots(nrows=2, sharex=True, figsize=(12, 6))
history_df = pd.DataFrame(history.history)
history_df["epoch"] = history.epoch
history_df.plot(x="epoch", y=["loss", "val_loss"], ax=ax0)
history_df.plot(x="epoch", y=["accuracy", "val_accuracy"], ax=ax1);



## Analiza

- Adam z domyślnymi parametrami i uczeniem np. 0.001 daje w wielu przypadkach zbieżność tak samo szybką lub nawet szybszą niż SGD z dobrze dobranymi eksperymentalnie parametrami dla szczególnego problemu

- Adam stosuje m.in. współczynnik uczenia lokalnie dla każdego neuronu dlatego dobieranie odpowiedniego współczynnika rzadko jest potrzebne

### Adam:     https://arxiv.org/abs/1412.6980


# Wynik działania sieci na zbiorze testowym

In [ ]:
y_predicted = np.argmax(model.predict(X_test), axis=-1)

fig, axes = plt.subplots(ncols=5, nrows=3, figsize=(12, 9))
for i, ax in enumerate(axes.ravel()):
    ax.imshow(scaler.inverse_transform(X_test[i]).reshape(8, 8), interpolation='nearest')
    ax.set_title("predicted label: %d\n true label: %d" % (y_predicted[i], y_test[i]))
    
print("test acc: %0.4f" % np.mean(y_predicted == y_test))

# Wpływ algorytmów uczenia na początkowy wybór wag

Zobaczmy teraz wpływ złego wyboru początkowego wag (initialization) na uczenie
sieci neuronowych.

Domyślnie warstwy Keras'a (Dense layers) wykorzystują strategię inicjalizacji <cite>"Glorot Uniform"[1][2][3]</cite> wag macierzy:

- każdy współczynnik wagi pochodzi z rozkładu jednostajnego na przedziale $[-\delta, \delta]$, gdzie  
  $$\delta \sim \frac{1}{\sqrt{n_{in} + n_{out}}}$$
  oraz $n_{in}$ i $n_{out}$ to liczba wejściowych i wyjściowych połączeń.

Strategia ta działa dobrze do inicjalizacji wag dla sieci neuronowych z funkcjami
aktywacji "tanh" lub "relu" i uczona za pomocą standardowego SGD.

Aby zobaczyć wpływ inicjalizacji wykorzystamy alternatywne metody inicjalizacji dla
dwuwarstwowej sieci z "tanh". Dla tego konkretnego przykładu wykorzystamy do inicjalizacji
wag rozkład normalny z dobranymi wartościami standardowego odchylenia.

[1]: [Xavier Glorot, Yoshua Bengio, Understanding the difficulty of training deep feedforward neural networks](http://proceedings.mlr.press/v9/glorot10a.html)

[2]: [What is the default weight initializer in keras](https://stackoverflow.com/questions/54011173/what-is-the-default-weight-initializer-in-keras)
 
[3]: [Weight Initialization in Neural Networks: A Journey From the Basics to Kaiming](https://towardsdatascience.com/weight-initialization-in-neural-networks-a-journey-from-the-basics-to-kaiming-954fb9b47c79)

In [ ]:
from tensorflow.keras import initializers

normal_init = initializers.TruncatedNormal(stddev=0.01)


model = Sequential()
model.add(Dense(hidden_dim, input_dim=input_dim, activation="tanh",
                kernel_initializer=normal_init))
model.add(Dense(hidden_dim, activation="tanh",
                kernel_initializer=normal_init))
model.add(Dense(output_dim, activation="softmax",
                kernel_initializer=normal_init))

model.compile(optimizer=optimizers.SGD(lr=0.1),
              loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.layers

Zobaczmy parametry pierwszej warstwy po inicjalizacji, ale przed uczeniem sieci. Zauważmy, że 'bias' jest ustawione na zero.

In [ ]:
model.layers[0].weights

In [ ]:
w = model.layers[0].weights[0].numpy()
w

In [ ]:
w.std()

In [ ]:
b = model.layers[0].weights[1].numpy()
b

In [ ]:
history = model.fit(X_train, Y_train, epochs=15, batch_size=32)

plt.figure(figsize=(12, 4))
plt.plot(history.history['loss'], label="Truncated Normal init")
plt.legend();

Zauważmy, że po uczeniu wagi zostają zaktualizowane i "bias" nie mają już wartości zero

In [ ]:
model.layers[0].weights

## Testy

- Wykonajmy poniższe schematy inicjalizacji i zobaczmy czy algorytm SGD nauczy lub nie naszą sieć neuronową

  - małe odchylenie np. `stddev=1e-3`
  - duże odchylenie `stddev=1` or `10`
  - inicjalizacja wszystkich wag na $0$ (stała)

- Czy metody uczenia SGD z momentem lub Adam lepiej poradzą sobie ze złą inicjalizacja wag?

In [ ]:
large_scale_init = initializers.TruncatedNormal(stddev=1)
small_scale_init = initializers.TruncatedNormal(stddev=1e-3)


optimizer_list = [
    ('SGD', optimizers.SGD(lr=0.1)),
    ('Adam', optimizers.Adam()),
    ('SGD + Nesterov momentum', optimizers.SGD(
            lr=0.1, momentum=0.9, nesterov=True)),
]

init_list = [
    ('glorot uniform init', 'glorot_uniform', '-'),
    ('small init scale', small_scale_init, '-'),
    ('large init scale', large_scale_init, '-'),
    ('zero init', 'zero', '--'),
]


for optimizer_name, optimizer in optimizer_list:
    print("Fitting with:", optimizer_name)
    plt.figure(figsize=(12, 6))
    for init_name, init, linestyle in init_list:
        model = Sequential()
        model.add(Dense(hidden_dim, input_dim=input_dim, activation="tanh",
                        kernel_initializer=init))
        model.add(Dense(hidden_dim, activation="tanh",
                        kernel_initializer=init))
        model.add(Dense(output_dim, activation="softmax",
                        kernel_initializer=init))

        model.compile(optimizer=optimizer,
                      loss='categorical_crossentropy')

        history = model.fit(X_train, Y_train,
                            epochs=10, batch_size=32, verbose=0)
        plt.plot(history.history['loss'], linestyle=linestyle,
                 label=init_name)

    plt.xlabel('# epochs')
    plt.ylabel('Training loss')
    plt.ylim(0, 6)
    plt.legend(loc='best');
    plt.title('Impact of initialization on convergence with %s'
              % optimizer_name)


## Analiza

- Jeśli sieć jest inicjalizowana na stała wartość zero, to aktywacja warstw ukrytych jest też ustawiana na zero,
  niezależnie od wartości wejściowych. Gradient jest też zawsze zero. Dlatego wszystkie algorytmy bazujące
  na spadku gradientu (SGD, Adam, ...) nie działają.

- Warto zaznaczyć, że model z softmax zachowuje się inaczej ...

- Dla sieci neuronowych kiedy inicjalizacja przebiegła dla małych wag. SGD ma duże problemy z powodu
  małego gradientu. Dodanie momentu może poprawić sytuacje, ale potrzeba dużo epok do nauczenia sieci
  
- Inicjalizaca wag na duże wartości powoduje zepsucie warstwy wyjścia (softmax). Sieć jest "pewna"
  swoich predykcji nawet jeśli są one kompletnie losowe

- 'Glorot uniform' wykorzystuje lepsze dostosowanie do wymiarów macierzy i zachowuje średnią aktywacje oraz 
  gradienty co powoduje lepszy proces uczenia

- Adam jest bardziej odporny na złą inicjalizację wag dzięki dobraniu współczynnika uczenia dla każdej wagi, ale
  i tak lepiej zachowuje się dla "dobrej" inicjalizacji wag
 

## Podumowanie

Na razie proszę zapamiętać, że jeśli sieć nie uczy się w ogóle 'loss' pozostaje taki sam, to

- upewnij się że wagi zostały dobrze dobrane
- sprawdź sieć warstwa po warstwie patrząc na gradient może to pomóc zidentyfikować "złą warstwę"
- wykorzystać Adam zamiast SGD

#### https://stackoverflow.com/questions/50033312/how-to-monitor-gradient-vanish-and-explosion-in-keras-with-tensorboard
